In [131]:
from xgboost import XGBClassifier
import xgboost as xgb

import pandas as pd 
import numpy as np
import copy as cp
from sklearn import preprocessing 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [132]:
data = pd.read_csv('./data/yanchen_data_process_for_train_201801311718.csv')
data = np.array(data)


In [133]:
input_data, label_data = pd.DataFrame(data[:, :-1]), pd.DataFrame(data[:, -1])
input_data, label_data

(          0   1   2    3   4   5   6   7   8   9   10  11  12  13  14    15  \
 0     103507   5   4  831   2   3   3   1   1   1   3   2   1   4   0    58   
 1     103507   5   5  831   2   3   3   0   1   1   3   2   1   4   0   232   
 2     103507   5   6  831   2   3   3   1   1   1   3   2   1   4   0   226   
 3     103507   5   7  831   2   3   3   1   3   1   1   2   1   4   0   286   
 4     103507   5   8  831   2   3   3   1   1   1   3   2   1   4   0   297   
 5     103507   5   9  831   2   3   3   1   1   1   1   2   1   4   0   355   
 6     103507   5  10  831   2   3   3   0   1   1   3   2   1   4   0   442   
 7     103507   5  11  831   2   3   3   1   1   1   3   2   1   4   0  1050   
 8     103507   6   2  831   2   3   4   0   1   1   3   2   1   4   0   993   
 9     103507   6   3  831   2   3   4   1   3   1   3   2   1   4   0   953   
 10    103507   6   4  831   2   3   3   1   1   1   1   2   1   4   0   650   
 11    103507   6   5  831   2   3   3  

<font color=orange size=6>随机分布得到验证集核训练集</font>

In [164]:
seed = 6
test_size = 0.1
x_train_part, x_validate, y_train_part, y_validate= train_test_split(input_data, label_data, test_size=test_size,
    random_state=seed)
x_train_part.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
4343,961362,4,11,692,2,5,4,1,1,1,3,2,1,4,0,67,47
945,291086,4,2,761,3,2,5,1,1,1,3,2,1,4,0,1043,526
484,219195,6,2,638,2,1,2,0,1,1,1,2,1,4,0,430,150
1592,384924,3,2,761,3,2,5,0,1,1,3,2,1,4,0,789,436
102,125403,6,4,761,2,2,5,1,1,1,1,2,1,4,2,197,361


<font color=orange size=6>以下开始XGBOOST调试过程</font>

In [169]:
dtrain = xgb.DMatrix(x_train_part, y_train_part)
dvalid = xgb.DMatrix(x_validate, y_validate)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]  
num_trees= 40000
params = {"objective": "reg:linear",  
              "eta": 0.05,  
              "max_depth": 20,  
              "subsample": 0.7,  
              "colsample_bytree": 0.7,  
              "silent": 1
              }   

<font color=blue size=5>将一部分作为验证集，一部分作为训练集</font>

In [170]:
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, early_stopping_rounds=50, verbose_eval=True)  


[0]	train-rmse:691.213	eval-rmse:573.962
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[1]	train-rmse:661.095	eval-rmse:547.679
[2]	train-rmse:633.429	eval-rmse:523.782
[3]	train-rmse:607.175	eval-rmse:498.948
[4]	train-rmse:581.584	eval-rmse:476.651
[5]	train-rmse:557.811	eval-rmse:455.36
[6]	train-rmse:535.839	eval-rmse:434.853
[7]	train-rmse:517.482	eval-rmse:420.148
[8]	train-rmse:496.328	eval-rmse:402.776
[9]	train-rmse:475.494	eval-rmse:384.54
[10]	train-rmse:461.662	eval-rmse:372.992
[11]	train-rmse:446.292	eval-rmse:360.49
[12]	train-rmse:429.718	eval-rmse:347.135
[13]	train-rmse:413.075	eval-rmse:333.725
[14]	train-rmse:397.109	eval-rmse:320.247
[15]	train-rmse:382.294	eval-rmse:307.251
[16]	train-rmse:367.793	eval-rmse:295.178
[17]	train-rmse:353.692	eval-rmse:285.054
[18]	train-rmse:340.662	eval-rmse:275.265
[19]	train-rmse:329.584	eval-rmse:267.518
[20]	train-rmse:318.332	eval-r

In [171]:
gbm.save_model('./models/model_201801311400')

In [177]:
test_data = pd.read_csv('./data/yanchen_data_process_for_test_201801311836.csv')
print(test_data.head(5))
test_data_df = pd.DataFrame(np.array(test_data))
test_data_dm = xgb.DMatrix(test_data_df)

        0  1   2    3  4  5  6  7  8  9  10  11  12  13  14   15   16
0  103507  7  10  831  2  3  3  1  1  1   1   2   1   4   0  186  180
1  124140  7  10  783  2  1  3  0  1  1   1   2   1   4   0  285  263
2  125403  7  10  761  2  2  7  1  1  1   1   2   1   4   2  328  156
3  136916  7  10  106  3  5  4  1  1  1   1   2   1   4   0  145  162
4  169673  7  10  682  2  1  1  0  1  1   1   2   1   4   2  219  152


<font color=orange size=6>开始预测</font>

In [178]:
y_pred = gbm.predict(test_data_dm).round()
y_pred

array([  186.,   251.,   150.,   159.,   139.,   265.,   146.,    85.,
         357.,   263.,   126.,  2076.,   184.,   150.,   325.,   123.,
         275.,   134.,   217.,   253.,   570.,   414.,   345.,   346.,
         287.,   197.,   399.,    80.,   147.,   399.,   113.,  2333.,
         182.,  2510.,   376.,  1101.,   536.,   605.,   507.,   138.,
         143.,   120.,   392.,    97.,   310.,   123.,   745.,   232.,
          86.,   144.,  1662.,   112.,   202.,   128.,  1934.,   422.,
         123.,   148.,   248.,   170.,   108.,   101.,   152.,   219.,
         191.,   343.,   196.,   151.,    75.,   379.,    46.,   165.,
         330.,   155.,   117.,   346.,   258.,   440.,   248.,   179.,
         274.,   161.,   434.,   166.,   376.,   380.,  1056.,   688.,
         126.,   204.,   116.,   168.,   663.,   101.,   308.,    82.,
         580.,   544.,   171.,   227.,   224.,   455.,   120.,   169.,
         217.,  2467.,   213.,   988.,   500.,   826.,  1067.,   117.,
      

In [179]:
pred_final = pd.read_csv("./data/yancheng_testA_20171225.csv")
for x in pred_final.index:
    pred_final['predict_quantity'][x] = y_pred[x]

C:\Users\jinda\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [180]:
pred_final.head(5)

,predict_date,class_id,predict_quantity
0,201711,103507,186.0
1,201711,124140,251.0
2,201711,125403,150.0
3,201711,136916,159.0
4,201711,169673,139.0


<font color=red size=6>保存最后数据</font>

In [181]:
pd.DataFrame.to_csv(pred_final, './results/result_201801312013.csv', index=False)